In [2]:
import joblib
import pandas as pd

In [3]:
df = pd.read_csv("../test_data/new leaky wave/S11_Data_combined.csv")
X = df.drop(columns=['dB(S(1,1)) []'])
complete_df = pd.DataFrame(columns=X.columns)

In [4]:
# Create all combinations of ranges 
ranges = {}
data_cols = list(X.columns)
for column in data_cols:
    ranges[column] = X[column].unique().tolist()

# Custom overrides
# ranges['inset_dist [mm]'] += [0.8, 1.2]
# ranges['L [mm]'] += [11.75, 12.25]
# ranges['W [mm]'] += [14.2, 14.4, 14.6, 15.0, 15.2, 15.4]
# ranges['W0 [mm]'] += [2.75, 3.25]
# ranges['y0 [mm]'] += [3.25, 3.75, 4.25, 4.75]

In [5]:
# Try cartesian product in pandas instead https://stackoverflow.com/a/13270110/3675086
ranges_df = pd.DataFrame(ranges[X.columns[0]], columns=[X.columns[0]])
for col in complete_df.columns[1:]:
    temp_df = pd.DataFrame(ranges[col], columns=[col])
    ranges_df = ranges_df.merge(temp_df, how='cross')

In [6]:
# Remove duplicates that are already calculated https://stackoverflow.com/a/47107164
df_all = ranges_df.merge(X.drop_duplicates(), on=X.columns.tolist(), how='left', indicator=True)
df_all = df_all[df_all['_merge'] == 'left_only']
df_all.drop(columns=['_merge'], inplace=True, axis=1)

In [8]:
model = joblib.load('../antenna_model.pkl')

predictions = model.predict(df_all)
df_all['prediction'] = predictions
df_all.to_pickle("all_predictions.pkl")